## Notebook 2
## Read and Write data to Azure SQL Managed Instance

This notebook emphasizes the connectivity, read and write operations to Azure SQL Managed Instance using the [SQL Spark connector](https://github.com/microsoft/sql-spark-connector).<br> This demo uses a Business Critical SQL MI with a readable secondary replica.

> For the purpose of this demo, the connectivity to SQL MI is
> established using the **public endpoint**. Port 3342 needs to be specified in this case.<br> For SQL MI connectivity without
> public endpoints, consider [injecting Databricks into SQL MI
> VNET](https://docs.microsoft.com/en-us/azure/databricks/administration-guide/cloud-configurations/azure/vnet-inject)
> or alternatively [connecting with VPN gateway or VNET
> peering](https://docs.microsoft.com/en-us/azure/azure-sql/managed-instance/connect-application-instance).

Import all the necessary Spark SQL functions used in this notebook for data transformation

In [3]:
from pyspark.sql.functions import to_date, concat_ws

The SQL MI connection and credential information are stored as secrets in ***Azure Key Vault***. The following snippet retrieves the secrets and stores them in variables.<br>
> The secrets are always Redacted when displayed in the notebook

In [5]:
sqlmiconnection = dbutils.secrets.get(scope = "sqlmi-kv-secrets", key = "sqlmiconn")
sqlmiuser = dbutils.secrets.get(scope = "sqlmi-kv-secrets", key = "sqlmiuser")
sqlmipwd = dbutils.secrets.get(scope = "sqlmi-kv-secrets", key = "sqlmipwd")
dbname = "Covid19datamart"
servername = "jdbc:sqlserver://" + sqlmiconnection
database_name = dbname
url = servername + ";" + "database_name=" + dbname + ";"

Read *DimCountry* table from SQL MI's ***ReadOnly replica*** and save it in a dataframe for transformations later in this notebook.

In [7]:
table_name = "[Covid19datamart].[dbo].[DimCountry]"

try:
  dfCountry = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("applicationintent", "ReadOnly") \
        .option("user", sqlmiuser) \
        .option("port", 3342) \
        .option("password", sqlmipwd).load()
except ValueError as error :
    print("Connector read failed", error)

display(dfCountry)

Read *StagingPredictedCovid19* table from SQL MI's ***ReadOnly replica*** and save it in a dataframe for transformations below. <br>This table contains the predicted values from the previous notebook.

In [9]:
table_name = "[Covid19datamart].[dbo].[StagingPredictedCovid19]"

try:
  dfStagingPredict = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("applicationintent", "ReadOnly") \
        .option("user", sqlmiuser) \
        .option("port", 3342) \
        .option("password", sqlmipwd).load().drop("Population")
except ValueError as error :
    print("Connector read failed", error)

display(dfStagingPredict)

Join the predicted data with country data (left join) to create a *denormalised dataset* for reporting and visualisation.

In [11]:
dfJoinedPredict = dfStagingPredict.join(dfCountry, dfStagingPredict.CountryName == dfCountry.CountryName,how='left').drop(dfCountry.CountryName)
display(dfJoinedPredict)

Include a date column with concatenated values from day, month and year;<br>
Cleanup the final dataset by dropping unwanted columns that are not required for reporting;

In [13]:
dfResult = dfJoinedPredict.withColumn("DateRecorded", to_date(concat_ws("-",dfJoinedPredict.Year,dfJoinedPredict.Month,dfJoinedPredict.Day)).cast('timestamp')) \
  .withColumnRenamed("Prediction","PredictedDeaths") \
  .drop("CountryMLIndex") \
  .drop("LoadDate") \
  .drop("DimCountryPK") \
  .drop("GeoID")

display(dfResult)

Write the final dataset to the table *FactCovid19* in SQL MI.<br>
Note the *applicationintent* is set to "*ReadWrite*" which is also the default when not specified.

In [15]:
table_name = "[Covid19datamart].[dbo].[FactCovid19]"

try:
  dfResult.write \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", sqlmiuser) \
        .option("port", 3342) \
        .option("password", sqlmipwd) \
        .option("applicationintent", "ReadWrite") \
        .mode("append") \
        .save()
except ValueError as error :
    print("Connector Write failed", error)


In [16]:
display(dfResult)

-- End of notebook --